<a href="https://colab.research.google.com/github/earendil94/SMLExam/blob/master/CaptionPreProcessing_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A quick overview

**Q) What still needs to be done regarding the RNN?**

A) We still need to find a proper way to sample an image and a caption.
I wanted to try to get the RNN to work alone but I am missing the proper
label if I use the text as an input. Moreover we need to define the learning procedure of our model

#### Useful stuff
https://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/

https://towardsdatascience.com/automatic-image-captioning-with-cnn-rnn-aae3cd442d83




In [7]:
#Import

import torch
from torch import nn
from torch import optim
import pandas as pd
from spacy.lang.en import English #I am not even sure we need it. In fact I believe we don't
import en_core_web_sm

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [136]:
#Loading captions file and creating the array

#TODO: this is going to be very rough for now
#Later:
#  Split our dataset into training/test
#  add a min number of repetitions in the training data to discard rare words 

import os

def prepare_data(path, input_file, output_file):
  input_path = os.path.join(path, input_file)
  output_path = os.path.join(path, output_file)
  df = pd.read_csv(input_path, sep = "|")
  captions_array = df[' comment']
  captions_array.fillna("", inplace=True)
  
  image_names = df["image_name"].values
  image_number = []
  for i in range(0, len(image_names)):
    image_number.append(image_names[i].split('.')[0])

  df.drop(labels=['image_name', ' comment_number'], axis=1, inplace=True)
  df.index = image_number
  df.to_csv(output_path, index_label="image_number")


prepare_data("/content/drive/My Drive/SML_Project/Chunk1", "results.csv", "clean.csv")

In [137]:
from torchtext.data import Field
from torchtext.data import TabularDataset

tokenize = lambda x : x.split()
TEXT = Field(sequential = True, tokenize = tokenize, lower=True, init_token='<start>', eos_token='<end>')
LABEL = Field(sequential=False, use_vocab=False)

td_datafields = [("image_number", LABEL ),
                 ("comment", TEXT)]

trn = TabularDataset(
              path="/content/drive/My Drive/SML_Project/Chunk1/clean.csv", # the root directory where the data lies
              format='csv',
              skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
              fields=td_datafields,
              #csv_reader_params={'delimiter':'\t'}
              )

#Build the vocabulary
trn.examples[0].comment

['two',
 'young',
 'guys',
 'with',
 'shaggy',
 'hair',
 'look',
 'at',
 'their',
 'hands',
 'while',
 'hanging',
 'out',
 'in',
 'the',
 'yard',
 '.']

In [139]:
#Building the vocabolary
TEXT.build_vocab(trn)
print(len(trn))
TEXT.vocab.itos[1]

158915


'<pad>'

In [147]:
from torchtext.data import Iterator
train_iter = Iterator(trn, batch_size=len(trn), device = -1)


for i in train_iter:
  cmt = i.comment
  img = i.image_number

cmt.T



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


tensor([[  2,   4,  10,  ...,   1,   1,   1],
        [  2,   4, 853,  ...,   1,   1,   1],
        [  2, 104,  15,  ...,   1,   1,   1],
        ...,
        [  2,   4, 172,  ...,   1,   1,   1],
        [  2,  16,  15,  ...,   1,   1,   1],
        [  2,   4,  69,  ...,   1,   1,   1]])

In [153]:
caption_indexed = cmt.T
print(caption_indexed[0])

for i in caption_indexed[0]:
  k = i.item()
  if k == 1:
    break
  print(TEXT.vocab.itos[k], end = " ")

print("\n", img[0].item())


for i in caption_indexed[1]:
  k = i.item()
  if k == 1:
    break
  print(TEXT.vocab.itos[k], end = " ")

print("\n", img[1].item())


caption_map = {}

#keys = 

#for i in range(1, len(trn)):



tensor([   2,    4,   10,    6,    4,   26,  281,   95,   20,    4,  602, 3125,
           8,    4,  428, 1050,    5,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1])
<start> a man in a black sweater sits at a desk typing on a computer keyboard . <end> 
 2203168780
<start> a singer in gray suit and black hat holds a guitar and sings into a microphone at an outside concert . <end> 
 4648495016


In [ ]:
#Actual run devices
embed_size = 100
hidden_size = 100
epochs = 3
path = "/content/drive/My Drive/SML_Project/Chunk1/results.csv"
learning_rate = 0.01


vocab_size = load_vocab(path=path)
decoder = DecoderRNN(embed_size=embed_size, hidden_size=hidden_size, vocab_size=vocab_size)
device = "cpu"

decoder.to(device)
criterion = nn.CrossEntropyLoss()
params = decoder.parameters()
optimizer = optim.Adam(lr = learning_rate)


Embedding(2, 3)

# THIS IS THE ACTUAL PART WE ARE INTERESTED IN

In [ ]:
import torch
from torch import nn
from torch import optim
import pandas as pd

#TODO: Right now this is basically the same exact class described in the towards data science article
#Should/Do we have to make any changes to this?
class DecoderRNN(nn.Module): 
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers = 1): #Here we define the layers
    super().__init__()
    self.embedding_captions_layer = nn.Embedding(vocab_size, embed_size)
    self.LSTM = nn.LSTM(input_size = embed_size, hidden_size = hidden_size, 
                        num_layers = num_layers, batch_first = TRUE)
    self.linear = nn.Linear(hidden_size, vocab_size)

  def forward(self, features, captions): #Notice that features will only be used when we will have the encoder images in input
    #captions = captions[:, :-1]
    embed = self.embedding_layer(captions)
    #A couple of comments here: torch.cat concatenates just like in unix
    #torch.unsqueeze instead transforms the tensor into a column vector (column since we specify 1 here)
    #embed = torch.cat((features.unsqueeze(1), embed), dim = 1) 
    lstm_outputs, _ = self.lstm(embed)
    out = self.linear(lstm_outputs)

In [242]:
#To put into a module
from torchtext.data import Field
from torchtext.data import TabularDataset
from torchtext.data import Iterator

def prepare_data(path, input_file, output_file):
  input_path = os.path.join(path, input_file)
  output_path = os.path.join(path, output_file)
  df = pd.read_csv(input_path, sep = "|")
  captions_array = df[' comment']
  captions_array.fillna("", inplace=True)
  
  image_names = df["image_name"].values
  image_number = []
  for i in range(0, len(image_names)):
    image_number.append(image_names[i].split('.')[0])

  df.drop(labels=['image_name', ' comment_number'], axis=1, inplace=True)
  df.index = image_number
  df.to_csv(output_path, index_label="image_number")
  return output_path

def build_vocab(path_to_caption_file, caption_file):
  output_path = prepare_data(path_to_caption_file, caption_file, "clean.csv")
  tokenize = lambda x : x.split()
  TEXT = Field(sequential = True, tokenize = tokenize, lower=True, init_token='<start>', eos_token='<end>')
  td_datafields = [("image_number", LABEL ),
                  ("comment", TEXT)]

  trn = TabularDataset(
              path=output_path, # the root directory where the data lies
              format='csv',
              skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
              fields=td_datafields,
              )
  
  TEXT.build_vocab(trn)
  return TEXT.vocab

def word_caption_to_index(path_to_caption_file, caption_file):
  '''
    This function takes as input the file containing the captions
    and returns a matrix of the captions indexed with respect to the inner
    vocabulary, as well as an array that can be used to map the indexed caption
    to the image it belongs to.
    @path_to_caption_file: the path to file containing the captions
    @caption_file: the name of the file containing the captions
  '''

  output_path = prepare_data(path_to_caption_file, caption_file, "clean.csv")
  tokenize = lambda x : x.split()
  TEXT = Field(sequential = True, tokenize = tokenize, lower=True, init_token='<start>', eos_token='<end>')
  LABEL = Field(sequential=False, use_vocab=False)

  td_datafields = [("image_number", LABEL ),
                  ("comment", TEXT)]

  trn = TabularDataset(
                path=output_path, # the root directory where the data lies
                format='csv',
                skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                fields=td_datafields
                )
  
  TEXT.build_vocab(trn)
  train_iter = Iterator(trn, batch_size=len(trn), device = -1)

  for i in train_iter: #TODO:I really havent got it how this still works honestly
    cmt = i.comment
    img = i.image_number

  return cmt.T, img

def vocab_as_dict(path_to_caption_file, caption_file):
  vocab = build_vocab(path_to_caption_file, caption_file)
  return vocab.stoi

def tensor_to_word(indexed_word, vocab):
  for i in indexed_word:
    k = i.item()
    if k == 1:
      break
    else:
      print(vocab.itos[k], end = " ")
    
  

def get_caption_from_image(caption_indexes, caption_refs, image_name):
  #If image name actually has the .jpg tail
  image_number = image_name.split('.')[0]
  caption_refs = (caption_refs == int(image_number))
  caption_refs = caption_refs.nonzero().T.numpy()[0].tolist()
  return caption_indexed[caption_refs]

In [228]:
caption_index, refs = word_caption_to_index("/content/drive/My Drive/SML_Project/Chunk1", "results.csv")
vox = build_vocab("/content/drive/My Drive/SML_Project/Chunk1", "results.csv")

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [244]:
caps = get_caption_from_image(caption_index, refs, '1000092795.jpg')
print(caps)

tensor([[    2,     4,    10,     6,     4,    29,    25,    35,     6,     4,
           715,     5,     3,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1],
        [    2,    16,   487,   816,   599, 12070,   140,     5,     3,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1

In [246]:
tensor_to_word(caps[2], vox)

<start> two men in green shirts are standing in a yard . <end> 